# Official policy publications of public organs

Collect publications from

 * https://lokaleregelgeving.overheid.nl
 * https://zoek.officielebekendmakingen.nl

regarding particular queries (see `queries` variables)

Publication HTMLs are simplified from unnecessary html/css/etc and converted to markdown.

In [3]:
#@title Set-up
import os
from tqdm.cli import tqdm
from multiprocessing import Pool
from urllib.parse import quote

tqdm.pandas()

import re
import sys
import json
import pickle
import zipfile
import requests
import warnings
import traceback
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse, quote
from IPython.display import display, HTML
from bs4.element import NavigableString, Comment

try:
  from markdownify import markdownify as md
except:
  !pip3 install markdownify
  from markdownify import markdownify as md

# to whatever convenient possibly empty folder:
# os.chdir('/ipfs-storage/ipfs/herbert/2024-12-17-joachim-bekkering/')
# os.chdir('/home/herbert/UG/2024-12-17-joachim-bekkering/')
os.chdir('/home/herbert/UG/collect-bibob-policies-and-tenders')

os.makedirs('cache', exist_ok=True)
os.makedirs('cache/bs', exist_ok=True)


def c_get(url, cache=True, **kwargs):
  """Return from cache or fetch"""
  filename = quote(url, safe='')
  if cache and os.path.exists(f'cache/{filename}.p3'):
    try:
      with open(f'cache/{filename}.p3', 'rb') as f:
        return pickle.load(f)
    except:
      pass
  response = requests.get(url, headers={
      'User-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
      'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
      'accept-encoding': 'gzip, deflate, br, zstd',
      'accept-language': 'en-US,en;q=0.8',
      'cache-control': 'max-age=0',
      'cookie': '.AspNetCore.Session=CfDJ8K8geeQnbxVAr4bBU5k2YcuEvFzwSoOlUOpP8xtnTF3QJLenbEhFwJcMRT27fuPtcCirsN7XoZ8LEg5YCKjQrzp4TcY6zCLhhivw86x7fseRL%2B3aH6OIsZmgHRHTgGT5iRGlMcQed9XuiCIx608gK9KlKyhJR9xOYaXIZWXTjRhZ',
      'priority': 'u=0, i',
      'sec-ch-u': '"Brave";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Linux"',
      'sec-fetch-dest': 'document',
      'sec-fetch-mode': 'navigate',
      'sec-fetch-site': 'none',
      'sec-fetch-user': '?1',
      'sec-gpc': '1',
      'upgrade-insecure-requests': '1',
  }, **kwargs)
  if cache and response.status_code == 200:
    with open(f'cache/{filename}.p3', 'wb') as f:
      pickle.dump(response, f)
  return response


def d_get(urls, n_jobs=10, cache=True, **kwargs):
  """HTTP get all `urls` of `pd.Series(...str...)` and return a series of
  responses with the same index"""
  if n_jobs == 1:
    return urls.progress_apply(lambda x: c_get(x, cache=cache, **kwargs))
  else:
    with Pool(n_jobs) as pool:
      promises = urls.apply(lambda x: pool.apply_async(c_get, (x,), dict(cache=cache, **kwargs)))
      responses = promises.progress_apply(lambda x: x.get())
  return responses


def iterate_pages(query, count, url, processing, **kwargs):
  """Iterate over pagination and returns dataframe of all hits"""
  def get_page_and_count(page):
    nonlocal query, count
    try:
      url_  = url.format(query=quote(query), count=count, page=page, **kwargs)
      response = c_get(url_, cache=False)
      doc = BeautifulSoup(response.text)
      h1 = next((h1 for h1 in doc.select('h1') if 'Geen resultaten gevonden' in h1.text), None)
      if h1 is not None:
        return response, 0
      h1 = next(h1 for h1 in doc.select('h1') if h1.text.strip().startswith('Zoekresultaten'))
      # n = int(h1.select('span.h1__sub')[0].text.split(' van de ')[1].strip().split(' ')[0])
      n = h1.select('span.h1__sub')[0].text.strip().lower()
      n = int(next(x for x in re.match(
          '(?:.* van de ([0-9]+) resultaten|([0-9]+) resultaten|(1) resultaat)',
          n).groups() if x is not None))
      return response, n
    except Exception as e:
      print(query, count, kwargs)
      print(url_)
      raise e

  def process(response):
    nonlocal base_url, url, processing, query, count
    try:
      doc = BeautifulSoup(response.text)
      hits = processing(doc)
      if len(hits) == 0:
        return pd.DataFrame()
      hits['search url'] = response.url
      hits['query'] = query
      hits['url'] = (base_url + hits['path'].apply(
          lambda x: x if x.startswith('/') else '/' + x))
    except Exception as e:
      print(query, count, kwargs)
      print(url_)
      raise e
    return hits

  base_url = urlparse(url.format(query=quote(query), count=count, page=1, **kwargs))
  base_url = f"{base_url.scheme}://{base_url.netloc}"

  response, n = get_page_and_count(1)
  urls = pd.Series({
    page: url.format(query=quote(query), count=count, page=page, **kwargs)
    for page in range(2, (n - 1) // count + 2)
  })
  hits = pd.concat(
      [process(response)] + d_get(urls, cache=False).progress_apply(process).tolist())
  hits['query'] = query
  for k, v in kwargs.items():
    hits[k] = v
  return hits

# Queries to search title and other fields for on govmntl. sites
queries = [
    'Bibob',
    'Bibob-beleid', 'Bibob-beleidsregel', 'Bibob-beleidsregels',
    'Bibob-beleidslijn', 'Bibob-beleidslijnen',
    'Wet Bibob',
    'bevordering integriteitsbeoordelingen door het openbaar bestuur',
    'Wet bevordering integriteitsbeoordelingen door het openbaar bestuur',
]

In [ ]:
#@title Find Bibob documents at local governments

rerun = True #@param {"type": "boolean"}

def parse_item(item):
  """Given a html-element for one hit, extract all its features (incl. url)"""
  result = {}
  for title in item.select('.item-header h2.result--title a'):
    if 'path' in result:
      raise ValueError(f'Multiple titles and URLs found for item {item}')
    result['path'] = title.attrs['href']
    result['title'] = title.text.strip()
    for subtitle in title.parent.select('p'):
      result['subtitle'] = subtitle.text.strip()
  for metadata in item.select('ul.list--metadata > li'):
    prefixes = {'Geldend ': 'Geldend', 'Uitgegeven door: ': 'Uitgever',
                'Dit is een toekomstige tekst! Geldend ': 'Geldend'}
    for prefix, field in prefixes.items():
      if metadata.text.startswith(prefix):
        break
    else:
      raise ValueError(f'Unknown metadata {metadata.text}')
    result[field] = metadata.text[len(prefix):].strip()

  return result

def extract_items(doc):
  """Hiven an html page, extract all the hits as dataframe"""
  items = pd.DataFrame([parse_item(item) for item in doc.select('.result--list > ul > li.item')])
  if 'Geldend' in items.columns:
    items = pd.concat([items, items['Geldend'].str.extract(
      r'^van(?:af)? (?P<van>.*) '
      r't\/m ?(?P<tot>(?:.(?!met terugwerkende kracht))*)'
      r'(?: met terugwerkende kracht vanaf (?P<terugwerkend>.*))?$'
    ).astype(str).fillna('')], axis=1)

    for field in ['van', 'tot', 'terugwerkend']:
      items[field] = pd.to_datetime(
        items[field].dropna(), format='%d-%m-%Y', errors='coerce')

  return items


def warn_with_traceback(message, category, filename, lineno, file=None, line=None):
  log = file if hasattr(file,'write') else sys.stderr
  traceback.print_stack(file=log)
  log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback

count = 100
url = (
  'https://lokaleregelgeving.overheid.nl/ZoekResultaat'
  '?{field}={query}'
  '&page={page}'
  '&count={count}')

if not rerun and os.path.exists('localgovernment-documents.p3'):
  localgovernment_documents = pd.read_pickle('localgovernment-documents.p3')
else:
  localgovernment_documents = pd.concat([
    iterate_pages(query, count, url, extract_items, field=field)
    for query in queries
    for field in ['titel', # 'tekst' # removed as per email 2025-02-25
                  ]
    for _ in [tqdm.pandas(desc=f'{field} - {query}')]
  ])

  localgovernment_documents.to_pickle('localgovernment-documents.p3')

localgovernment_documents_unique = localgovernment_documents.groupby('path').agg({
  'query': ','.join, 'field': ','.join, 'url': 'first', 'title': 'first',
  'Geldend': 'first', 'Uitgever': 'first', 'van': 'first', 'tot': 'first',
  'terugwerkend': 'first'
})


In [7]:
localgovernment_documents = pd.read_excel('localgovernment-documents (bewerkt v2).xlsx', sheet_name='Opgeschoond')
localgovernment_documents_unique = localgovernment_documents.groupby('path').agg({
  'query': ','.join, 'field': ','.join, 'url': 'first', 'title': 'first',
  'Geldend': 'first', 'Uitgever': 'first', 'van': 'first', 'tot': 'first',
  'terugwerkend': 'first', 'title': 'first',
  #'Datum publicatie': 'first', 'Jaargang en nummer': 'first', 'Organisatie': 'first',
})

In [6]:
#@title Fetch local government documents

tqdm.pandas(desc=None)
localgovernment_responses = d_get(localgovernment_documents_unique['url'], n_jobs=10)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 902/902 [01:06<00:00, 13.58it/s]


In [ ]:
#@title Find Bibob documents at other official bodies

rerun = True #@param {"type": "boolean"}

def parse_item(item):
  result = {}
  for title in item.select('h2.result--title a'):
    if 'path' in result:
      raise ValueError(f'Multiple titles and URLs found for item {item}')
    result['path'] = title.attrs['href']
    result['title'] = title.text.strip()
    for subtitle in title.parent.parent.select('p'):
      result['subtitle'] = subtitle.text.strip()

  for subtitle in item.select('h2.result--subtitle a'):
    if 'subtitle' in result:
      raise ValueError(f'Multiple titles and URLs found for item {item}')
    result['subtitle'] = subtitle.text.strip()

  for metadata in item.select('dl.dl > div'):
    field = metadata.select('dt')[0].text.strip()
    value = metadata.select('dd')[0].text.strip()
    result[field] = value

  for a in item.select('ul.result--actions > li > a'):
    if a.text.strip() != 'Download PDF':
      raise ValueError(f'Unknown action {metadata.text}')
    result['pdf'] = a.attrs['href']

  return result


def extract_items(doc):
  items = pd.DataFrame([
      parse_item(item) for item in doc.select('.result--list > ul > li')])
  return items


count = 100
url = (
  'https://zoek.officielebekendmakingen.nl/resultaten'
  '?q=(c.product-area==%22officielepublicaties%22)'
  'and'
  '('
    '(w.publicatienaam==%22Staatsblad%22)or'
    '(w.publicatienaam==%22Staatscourant%22)or'
    '(w.publicatienaam==%22Blad%20gemeenschappelijke%20regeling%22)'
  ')'
  'and'
  '({field_prefix}.{field}=%22{query}%22)'
  'and'
  '('
    '(dt.type==%22beleidsregel%22)or'
    '(dt.type==%22circulaire%22)or'
    '(dt.type==%22convenant%22)or'
    '(dt.type==%22interne%20regeling%22)or'
    '(dt.type==%22overige%20overheidsinformatie%22)'
  ')'
  '&zv={query}'
  '&pg={count}'
  '&pagina={page}'
  # '&col=AlleBekendmakingen'
  # '&col=Staatsblad,Staatscourant,BladGemeenschappelijkeregeling'
)

# circulaire", "convenant", "interne regeling" en "overige overheidsinformatie"

if not rerun and os.path.exists('other-documents.p3'):
  other_documents = pd.read_pickle('other-documents.p3')
else:
  other_documents = pd.concat([
    iterate_pages(query, count, url, extract_items, field=field, field_prefix="dt" if field == "title" else "cql")
    for query in queries
    for field in ['textAndIndexes', 'title', ]
    for _ in [tqdm.pandas(desc=f'{field} - {query}')]
  ])
  other_documents.to_pickle('other-documents.p3')

other_documents_unique = other_documents.groupby('path').agg({
  'query': ','.join, 'field': ','.join, 'url': 'first', 'title': 'first',
  'pdf': 'first', 'Datum publicatie': 'first', 'Jaargang en nummer': 'first',
  'Organisatie': 'first',
})

In [9]:
other_documents = pd.read_excel('other-documents1 - bewerkt.xlsx', sheet_name='Opgeschoond')
other_documents_unique = other_documents.groupby('path').agg({
  'query': ','.join, 'field': ','.join, 'url': 'first', 'title': 'first',
  'pdf': 'first', 'Datum publicatie': ','.join, 'Jaargang en nummer': 'first',
  'Organisatie': 'first', 'subtitle': 'first',
})

In [10]:
#@title Fetch other official bodies' documents

tqdm.pandas(desc=None)
other_responses = d_get(other_documents_unique['url'], n_jobs=1, timeout=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:05<00:00,  2.27it/s]


In [11]:
#@title Failures (non-200) ?

pd.concat([
  localgovernment_responses.apply(lambda x: x.status_code).rename('status code').value_counts(),
  other_responses.apply(lambda x: x.status_code).rename('status code').value_counts()
], keys=['local', 'other'])

       status code
local  200            902
other  200             13
Name: count, dtype: int64

In [12]:
#@title parse HTML

rerun = True #@param {"type": "boolean"}

if not rerun and os.path.exists('localgovernment-docs-bs.p3'):
  localgovernment_docs = pd.read_pickle('localgovernment-docs-bs.p3')
else:
  localgovernment_docs = localgovernment_responses.progress_apply(lambda x: BeautifulSoup(x.text))
  localgovernment_docs.to_pickle('localgovernment-docs-bs.p3')


if not rerun and os.path.exists('other-docs-bs.p3'):
  other_docs = pd.read_pickle('other-docs-bs.p3')
else:
  other_docs = other_responses.progress_apply(lambda x: BeautifulSoup(x.text))
  other_docs.to_pickle('other-docs-bs.p3')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 19.57it/s]


In [14]:
#@title Simplify HTML by removing unessential attributes, spans, divs, etc

images_path = './images/'
os.makedirs(images_path, exist_ok=True)

def skip_div(node, skip_paragraphs=False):
  if isinstance(node, Comment):
    pass
  elif (node.name in {'div', 'section', 'container', 'span', 'br', 'figure',
                      'picture', 'blockquote', }
        or (skip_paragraphs and node.name in {'p'})):
    for element in node.children:
      yield from skip_div(element, skip_paragraphs=skip_paragraphs)
  elif isinstance(node, NavigableString):
    if node.strip() != '':
      yield re.sub(r'(?:\n| |\t)+', ' ', node.strip()) + ' '
  elif node.name in {'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'ul', 'ol', 'hr',
                     'a', 'em', 'strong', 'li', 'i', 'b', 'table', 'td', 'th',
                     'tr', 'colgroup', 'col', 'tbody', 'thead', 'img', 'cite',
                     'caption', 'object', 'dl', 'dt', 'dd', 'sup', 'sub', 'dfn',
                     'figcaption'}:
    new_children = [child
                    for child in node.children
                    for child in skip_div(
                        child, skip_paragraphs=node.name in {'li', 'ol', 'ul', 'table', 'tr', 'td', 'dd', 'dt', 'dl'})]
    while True:
      for child in node.children:
        child.extract()
        break
      else:
        break
    for child in new_children:
      node.append(child)
    if node.name == 'img':
      # response = requests.get(node.attrs['src'])
      # print(node.attrs['src'])
      node.attrs = {'src': node.attrs['src']}
    else:
      node.attrs = {}
    if node.text.strip() != '':
      yield node
  else:
    raise ValueError(f'Unknown tag {node.name}: {node}')

def simple_html(response):
  """Remove superfluous HTML and CSS, particularly overly nested stuff"""
  doc = BeautifulSoup(response.text)
  result = BeautifulSoup('', 'html.parser')
  if len(doc.select('#broodtekst')) > 0:
    for child in skip_div(doc.select('#broodtekst')[0]):
      result.append(child)
  else:
    for child in skip_div(doc.select('#PaginaContainer')[0]):
      result.append(child)
  return result


localgovernment_simple_html = localgovernment_responses.progress_apply(simple_html)
other_simple_html = other_responses.progress_apply(simple_html)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 16.40it/s]


In [19]:
#@markdown ## Keyword search

keyword_exps = {
    'subsidie': r'subsidie(?:s)?',
    'vergunning': r'vergunning(?:en)?',
    'beschikking': r'beschikking(?:en)?',
    'aanbesteding': r'aanbesteding(?:en)?',
    'overheidsopdracht': r'overheidsopdracht(?:en)?',
    'vastgoedtransactie': r'vastgoedtransactie(?:s)?',
    'openhouse': r'open(?: |-|)house'}

other_keywords = other_simple_html.apply(str).apply(md).apply(lambda text:
  pd.Series({k: len(re.findall(r'\b' + exp + r'\b', text)) for k, exp in keyword_exps.items()}))

localgovernment_keywords = localgovernment_simple_html.apply(str).apply(md).apply(lambda text:
  pd.Series({k: len(re.findall(r'\b' + exp + r'\b', text)) for k, exp in keyword_exps.items()}))


pd.concat([other_documents_unique, other_keywords], axis=1).to_excel('other-keywords.xlsx')
pd.concat([localgovernment_documents_unique, localgovernment_keywords], axis=1).to_excel('localgovernment-keywords.xlsx')

In [16]:
#@title Terms bibob-x

#@markdown Any occurrence of a term with bibob in it.

terms = {
    x for x in other_simple_html.progress_apply(lambda x: x.text).str.lower().str.findall('(bibob-[a-zA-Z0-9]+)')
    for x in x
} | {
  x for x in other_documents['title'].str.lower().str.findall('(bibob-[a-zA-Z0-9]+)')
    for x in x
} | {
    x for x in localgovernment_simple_html.progress_apply(lambda x: x.text).str.lower().str.findall('(bibob-[a-zA-Z0-9]+)')
    for x in x
} | {
  x for x in localgovernment_documents['title'].str.lower().str.findall('(bibob-[a-zA-Z0-9]+)')
    for x in x
}
terms

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 902/902 [00:00<00:00, 2301.59it/s]


{'bibob-aandachtsbranche',
 'bibob-aandachtsgebied',
 'bibob-aangelegenheid',
 'bibob-aanvraag',
 'bibob-aanvraagformulier',
 'bibob-aanvraagformulieren',
 'bibob-abel',
 'bibob-advies',
 'bibob-adviesaan',
 'bibob-adviesaanvraag',
 'bibob-adviesprocedure',
 'bibob-adviezen',
 'bibob-adviseur',
 'bibob-adviseurs',
 'bibob-aspecten',
 'bibob-be',
 'bibob-beding',
 'bibob-begeleidingscommissie',
 'bibob-beleid',
 'bibob-beleidregel',
 'bibob-beleidsegels',
 'bibob-beleidslijn',
 'bibob-beleidslijnen',
 'bibob-beleidslijngemeente',
 'bibob-beleidsregel',
 'bibob-beleidsregels',
 'bibob-beleidsterreinen',
 'bibob-benadering',
 'bibob-beoordeling',
 'bibob-bepaling',
 'bibob-bescheiden',
 'bibob-besluit',
 'bibob-besluitvorming',
 'bibob-betrokkene',
 'bibob-bevoegdheden',
 'bibob-bevoegdheid',
 'bibob-bureau',
 'bibob-categorie',
 'bibob-check',
 'bibob-clausule',
 'bibob-co',
 'bibob-conclusie',
 'bibob-criteria',
 'bibob-documenten',
 'bibob-dossier',
 'bibob-dossiers',
 'bibob-duo',
 'b

In [18]:
#@title Archive as zip
with pd.ExcelWriter('localgovernment-documents.xlsx', engine='xlsxwriter') as writer:
  localgovernment_documents[localgovernment_documents['field'].str.contains('titel')].to_excel(writer, sheet_name='Titel', index=False)
  localgovernment_documents[~localgovernment_documents['field'].str.contains('titel')].to_excel(writer, sheet_name='Niet in titel', index=False)

with pd.ExcelWriter('other-documents.xlsx', engine='xlsxwriter') as writer:
  other_documents[other_documents['field'].str.contains('title')].to_excel(writer, sheet_name='Titel', index=False)
  other_documents[~other_documents['field'].str.contains('title')].to_excel(writer, sheet_name='Niet in titel', index=False)

with zipfile.ZipFile('bibob-policies.zip', 'w') as zf:
  zf.write('localgovernment-documents.xlsx')
  zf.write('other-documents.xlsx')
  for path, response in other_responses.loc[
    other_documents.query('field.str.contains("title")')['path'].unique()
  ].items():
    path = path.strip(' /')
    zf.writestr(f"other/raw/html/{path}", response.text)
  for path, response in d_get(other_documents[
    other_documents['field'].str.contains('title')
  ].groupby('path').first()['url'].str.replace('\.html?$', '.pdf', regex=True)).items():
    path = re.sub('\.html?$', '', path.strip(' /')) + '.pdf'
    zf.writestr(f"other/raw/pdf/{path}", response.content)

  for path, response in localgovernment_responses.loc[
    localgovernment_documents.query('field.str.contains("titel")')['path'].unique()
  ].items():
    path = path.strip(' /') + '.html'
    zf.writestr(f"localgovernment/raw/{path}", response.text)

  for path, html in other_simple_html.loc[other_documents.query('field.str.contains("title")')['path'].unique()].items():
    txt = str(html)
    path = path.strip(' /')
    zf.writestr(f"other/simplified/html/{path}", txt)
    path = re.sub('\.html?$', '', path) + '.md'
    zf.writestr(f"other/simplified/markdown/{path}", md(txt))
  for path, html in localgovernment_simple_html.loc[
    localgovernment_documents.query('field.str.contains("titel")')['path'].unique()
  ].items():
    txt = str(html)
    path = path.strip(' /')
    zf.writestr(f"localgovernment/simplified/html/{path}", txt)
    path = re.sub('\.html?$', '', path) + '.md'
    zf.writestr(f"localgovernment/simplified/markdown/{path}", md(txt))
!echo scp il-c03:`pwd`/bibob-policies.zip .

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.12it/s]


scp il-c03:/home/herbert/UG/collect-bibob-policies-and-tenders/bibob-policies.zip .
